In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tcn import TCN, tcn_full_summary

In [ ]:
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
# Load the data from CSV files
input_df = pd.read_csv("./subject1_walking_15_17_11_2_MFthr5.csv")
gt_df = pd.read_csv("./acc_walking_forearm_18455_s1.csv")
input_np = np.array(input_df)
gt_np = np.array(gt_df)

# get spine length to normalize
neck = input_np[:,6:9]
center = input_np[:,9:]
spine_len = np.sqrt(np.sum((neck-center)**2, axis=1))
plt.plot(spine_len)

# second derivate
input_np_norm_by_spine = input_np[:,:6]/(np.array([spine_len]*6).T )
input_np_derivate = np.gradient(input_np_norm_by_spine, axis=0)
input_np_derivate_2 = np.gradient(input_np_derivate, axis=0)



In [ ]:
def standardize_data(data):
    # Calculate the mean along the first axis (each column)
    mean = np.mean(data, axis=0)
    # Calculate the standard deviation along the first axis (each column)
    std_dev = np.std(data, axis=0)
    # Standardize the data
    standardized_data = (data - mean) / std_dev
    return standardized_data

# input_np_std = standardize_data(input_np)
input_np_std = standardize_data(input_np_derivate_2)
gt_np_std = standardize_data(gt_np)


window_size = 60

input_data_windows = np.array([input_np_std[i-window_size:i,:] for i in range(window_size, input_np_std.shape[0])])
target_data_windows = gt_np_std[window_size:]



In [ ]:
# define TCN model
inputs = tf.keras.Input(shape=(window_size, 6))    

x = TCN(nb_filters=64,
                kernel_size=10,
                nb_stacks=2,
                dilations=(1, 2, 4, 8, 16, 32, 64),
                padding='same',
                use_skip_connections=True,
                dropout_rate=0.3,
                return_sequences=False,                 
                kernel_initializer='he_normal',
                use_batch_norm=False,
                use_layer_norm=False,
                use_weight_norm=False,
                activation="tanh")(inputs)        
outputs = tf.keras.layers.Dense(3, dtype=tf.float32)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='mse')

model.summary()

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 10 == 0:
            clear_output(wait=True)
            
callbacks = [
                DisplayCallback(),
                tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", verbose=0, patience=5),
                tf.keras.callbacks.ModelCheckpoint("./output/V8.h5", monitor="mse", mode="min", save_best_only=False)
            ]

def scale_fn(x):
    return 1/(2.**(x-1))

In [ ]:
# set hyper-parameters
lr = tfa.optimizers.CyclicalLearningRate(1e-4, 1e-2, step_size=20, scale_fn=scale_fn, scale_mode="cycle")
optimizer = tfa.optimizers.Lookahead(tfa.optimizers.AdaBelief(learning_rate=lr))
optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
model.compile(optimizer=optimizer, loss="mse")


In [ ]:
history = model.fit(input_data_windows, target_data_windows, epochs=1000, validation_split=0.2, callbacks=callbacks)  

In [ ]:
# # Validation: last 20% of training data  (Original)
# val_input = input_data_windows[int(input_df.shape[0]*0.8):]
# val_target = target_data_windows[int(input_df.shape[0]*0.8):]

# print(val_input.shape)
# print(val_target.shape)

In [ ]:
# Validation
val_input_df = pd.read_csv("./subject1_running_15_17_MFthr5.csv")
val_gt_df = pd.read_csv("./acc_running_forearm_7200_s1.csv")

val_input_np = np.array(val_input_df)
val_gt_np = np.array(val_gt_df)

# mean and std from training dataset
# val_input_np_std = (val_input_np-input_mean)/input_std
# val_gt_np_std = (val_gt_np- gt_mean)/gt_std

# mean and std from validation dataset
val_input_np_std = standardize_data(val_input_np)
val_gt_np_std = standardize_data(val_gt_np)


val_input_data_windows = np.array([val_input_np_std[i-window_size:i,:] for i in range(window_size, val_input_np_std.shape[0])])
val_target_data_windows = val_gt_np_std[window_size:]

val_input = val_input_data_windows[:int(val_input_df.shape[0]*0.5)]
val_target = val_target_data_windows[:int(val_input_df.shape[0]*0.5)]

In [ ]:
val_predictions = model.predict(val_input)

In [ ]:
import matplotlib.pyplot as plt

# Plot the actual vs predicted values for a specific target column
# target_column = 0  # Change this to the column you want to visualize
start=0
end=len(val_target)
# end=60
for target_column in range(3):
    plt.figure(figsize=(10,6))
    plt.plot(val_target[start:end, target_column], label='Actual')
    plt.plot(val_predictions[start:end, target_column], label='Predicted')
    plt.legend()
    plt.show()